In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import mean_squared_error

In [ ]:
sample_df=pd.read_csv('sample_4E0BhPN.csv')
test_df=pd.read_csv('test_6QvDdzb.csv')
train_df=pd.read_csv('train_E1GspfA.csv')

# Checking null values

In [ ]:
train_df.isnull().sum()

date      0
hour      0
demand    0
dtype: int64

In [ ]:
train_df.duplicated().sum()

0

# Feature Engineering

In [ ]:
def data_enginerring(x_t):
    x_t=x_t.copy()
    x_t["date"]=pd.to_datetime(x_t['date'])
    x_t['year']=x_t["date"].dt.year
    x_t['month']=x_t["date"].dt.month
    x_t['week']=x_t["date"].dt.week
    x_t['day']=x_t["date"].dt.day
    return x_t
    

In [ ]:
train_df=data_enginerring(train_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [ ]:
X=train_df[['hour', 'year', 'month', 'week', 'day']]
y=train_df['demand']

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly=PolynomialFeatures(2)

In [ ]:
X=poly.fit_transform(X)

# train test data spliting

In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)


# Ensamble Methods

In [ ]:

ereg = VotingRegressor(estimators=[('gb', GradientBoostingRegressor()), ('knn', KNeighborsRegressor()),('randomfore',RandomForestRegressor(n_estimators=800,random_state=30,ccp_alpha=0.01)),('histogram',HistGradientBoostingRegressor())])

ereg.fit(X_train, y_train)
print(f'train RMSE: {mean_squared_error(y_train,ereg.predict(X_train),squared=False)}')
print(f'test RMSE: {mean_squared_error(y_test,ereg.predict(X_test),squared=False)}')     # test base on unknown data

train RMSE: 25.394927931107592
test RMSE: 32.22403972091486


In [ ]:

ereg.fit(X, y)    # use all dataset because... 


VotingRegressor(estimators=[('gb', GradientBoostingRegressor()),
                            ('knn', KNeighborsRegressor()),
                            ('randomfore',
                             RandomForestRegressor(ccp_alpha=0.01,
                                                   n_estimators=800,
                                                   random_state=30)),
                            ('histogram', HistGradientBoostingRegressor())])

# test data convert into trainable type

In [ ]:
new_test=data_enginerring(test_df)
new_test=new_test.drop(axis=0,columns='date',)
new_test=poly.fit_transform(new_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [ ]:
new_test

array([[1.000e+00, 0.000e+00, 2.021e+03, ..., 8.100e+01, 9.000e+00,
        1.000e+00],
       [1.000e+00, 1.000e+00, 2.021e+03, ..., 8.100e+01, 9.000e+00,
        1.000e+00],
       [1.000e+00, 2.000e+00, 2.021e+03, ..., 8.100e+01, 9.000e+00,
        1.000e+00],
       ...,
       [1.000e+00, 2.100e+01, 2.022e+03, ..., 1.690e+02, 3.640e+02,
        7.840e+02],
       [1.000e+00, 2.200e+01, 2.022e+03, ..., 1.690e+02, 3.640e+02,
        7.840e+02],
       [1.000e+00, 2.300e+01, 2.022e+03, ..., 1.690e+02, 3.640e+02,
        7.840e+02]])

In [ ]:
# pridictin data and add into dataframe

test_df['demand']=ereg.predict(new_test)
test_df['demand']=test_df['demand'].astype(np.int64,)
test_df.head()

,date,hour,demand
0,2021-03-01,0,72
1,2021-03-01,1,63
2,2021-03-01,2,59
3,2021-03-01,3,55
4,2021-03-01,5,59


In [ ]:
test_df.to_csv('submission_essamble2_stander_scale.csv',index=False)